# #0

In [46]:
# ---- ¿Que año, mes y # de iteraciones quieres correr? ----
anyo = '2020'
mes = 'Abril'
iters = 100

In [2]:
# ---- Librerías ---- 
import pandas as pd 
import numpy as np
import scipy
from scipy.spatial import distance
import csv 
import warnings
import unicodedata
warnings.filterwarnings('ignore')

In [3]:
#Obtener la distancia promedio de un grupo de m puntos
def get_avg_distance(vectors):    
    output = distance.pdist(vectors, metric='euclidean')
    average = sum(output)/len(output)
    return average


In [44]:
# Leer CSV posiciones vectoriales
df_1 = pd.read_csv('embeddings_regiones/1.csv', index_col=1)
df_2 = pd.read_csv('embeddings_regiones/2.csv', index_col=1)
df_3 = pd.read_csv('embeddings_regiones/3.csv', index_col=1)
df_4 = pd.read_csv('embeddings_regiones/4.csv', index_col=1)
df_5 = pd.read_csv('embeddings_regiones/5.csv', index_col=1)
df_6 = pd.read_csv('embeddings_regiones/6.csv', index_col=1)
df_7 = pd.read_csv('embeddings_regiones/7.csv', index_col=1)
df_8 = pd.read_csv('embeddings_regiones/8.csv', index_col=1)
df_9 = pd.read_csv('embeddings_regiones/9.csv', index_col=1)
df_10 = pd.read_csv('embeddings_regiones/10.csv', index_col=1)
df_11 = pd.read_csv('embeddings_regiones/11.csv', index_col=1)
df_12 = pd.read_csv('embeddings_regiones/12.csv', index_col=1)
df_13 = pd.read_csv('embeddings_regiones/13.csv', index_col=1)
df_14 = pd.read_csv('embeddings_regiones/14.csv', index_col=1)
df_15 = pd.read_csv('embeddings_regiones/15.csv', index_col=1)
df_16 = pd.read_csv('embeddings_regiones/16.csv', index_col=1)

dfs_embeddings = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_14, df_15, df_16]

for df_ in dfs_embeddings:
    cols = df_.select_dtypes(include=[np.object]).columns
    df_[cols] =df_[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
    df_.index = df_.index.str.replace(u"Ñ","N")

In [5]:
# ---- DB Apellidos Paternos ---- 
df_paternos = pd.read_excel('data publica/apellidos_paternos.xlsx', index_col=0, usecols = 'A:D',engine="openpyxl")


In [6]:
# ---- DB Employees ---- 
df_whole = pd.read_csv('data publica/unidos/'+ mes + anyo + '.csv', index_col=0)


In [7]:
## ---------- Remover tildes de los apellidos y pasar a mayuscula ---------- ##
cols = df_paternos.select_dtypes(include=[np.object]).columns
df_paternos[cols] =df_paternos[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
cols = df_whole.select_dtypes(include=[np.object]).columns
df_whole[cols] =df_whole[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df_whole['Paterno'] = df_whole['Paterno'].str.upper()
df_whole['Materno'] = df_whole['Materno'].str.upper()

In [8]:
## ---------- Apellidos Paternos x Región ---------- ##
pd.options.display.float_format = '{:,.2f}'.format 
df_paternos['AP PATERNO'] = df_paternos['AP PATERNO'].str.upper()
df_paternos_region = df_paternos[df_paternos['AP PATERNO'].notna()]
df_paternos_region = df_paternos_region.groupby(['REG','AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_region = df_paternos_region[df_paternos_region['CANTIDAD'] > 1]
df_paternos_region['CANTIDAD'] = df_paternos_region['CANTIDAD'].astype(int)
## ---------- Apellidos Paternos en Chile ---------- ##
df_paternos_region = df_paternos_region.reset_index(level=['REG'])
df_paternos_total = df_paternos_region.groupby(['AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_total['PORCENTAJE'] = (df_paternos_total['CANTIDAD']/df_paternos_total['CANTIDAD'].sum())
df_paternos_total['CANTIDAD'] = df_paternos_total['CANTIDAD'].astype(int)
df_paternos_total = df_paternos_total.dropna()
df_paternos_total = df_paternos_total[df_paternos_total['CANTIDAD'] > 1]

In [40]:
df_organism_ = pd.read_csv('data publica/Acronimos Organismos.csv', encoding='ISO-8859-1', delimiter=';')
df_organism_


,Codigo_org,idOrg,Organismo,Codigo_padre,Padre_org,Region,Municipalidad,direccion,telefono,url_organismo,...,tipo_cuenta,banco,url_sai,fax,Ingresa,obligadorecibir_sai,organismo_autonomo,interopera,tiene_ta,fecha_ta
0,MA031,327550,Asociación de Municipios Ciudad Sur,ASOC,Asociación Municipal,Sin especificar,Sin especificar,56 9 7854 6853,56 9 7854 6853,http://www.municipiosciudadsur.cl/,...,NaN,NaN,http://,NaN,No ingresa,SÍ,NO,NO,SÍ,2018/01/01
1,CM038,58864,Corporación Municipal de Iquique,CM000,Corporaciones Municipales,Región de Tarapacá,IQUIQUE,NaN,NaN,http://sitio.cormudesi.cl/,...,NaN,NaN,http://solicitudes.cormudesi.cl/,NaN,No ingresa,SÍ,NO,NO,NO,NaN
2,UN002,1082,Universidad Arturo Prat (UNAP),UN000,Universidades,Región de Tarapacá,IQUIQUE,AV. ARTURO PRAT 2120 IQUIQUE,(57) 2 526811,http://www.unap.cl,...,NaN,NaN,http://transparencia.unap.cl/pasiva/web/index....,NaN,No ingresa,SÍ,NO,NO,SÍ,NaN
3,AO017,871,Servicio de Salud Iquique,AO000,Salud,Región de Tarapacá,IQUIQUE,Aníbal Pinto N 815,56 57 2409532,http://www.saludiquique.cl/,...,Cuenta Corriente,Bancoestado,http://transparencia.redsalud.gob.cl/transpare...,NaN,Ingresa,SÍ,NO,NO,SÍ,2018/02/23
4,AO040,56251,SEREMI de Salud de Tarapacá,AO000,Salud,Región de Tarapacá,IQUIQUE,Esmeralda 475 Iquique,(57) 2404655 (57) 2404651,http://www.seremisaludtarapaca.cl,...,NaN,NaN,http://www.portaltransparencia.cl/PortalPdT/we...,NaN,Ingresa,SÍ,NO,NO,SÍ,2018/01/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,MA062,330653,Asociación de Municipios del Sur de Chiloé (AM...,ASOC,Asociación Municipal,NaN,NaN,NaN,NaN,http://,...,NaN,NaN,http://,NaN,No ingresa,SÍ,NO,NO,NO,NaN
976,AE024,330510,TRIBUNAL TRIBUTARIO Y ADUANERO DE LA REGIÓN DE...,AE000,Hacienda,NaN,NaN,NaN,NaN,http://,...,NaN,NaN,http://,NaN,No ingresa,NO,NO,NO,NO,NaN
977,CO006,330600,Consejo Fiscal Autónomo,CO006,Consejo Fiscal Autónomo,NaN,NaN,NaN,NaN,http://,...,NaN,NaN,http://,NaN,No ingresa,SÍ,NO,NO,SÍ,2020/06/26
978,AJ027,330800,Servicio Local de Educación Pública Atacama,AJ000,Educación,NaN,NaN,NaN,NaN,http://,...,NaN,NaN,http://,NaN,No ingresa,NO,NO,NO,SÍ,2020/09/09


In [41]:
# Método MonteCarlo 
def MonteCarlo(population, sample_number, weights_column, iterations, code_region, current_distance):
    
    results = {'higher': 0, 'lower': 0, 'equal': 0, 'sum': 0}
    
    for i in range(0, iterations): #numero de iteraciones
        
        #esto saca un random example de la poblacion (general), le damos un "N" y un peso específico a cada apellido
        random_weighted_sample = population.sample(n = sample_number, replace = True, 
                                                   weights= weights_column, axis = 0)
        
        #transformar apellidos en matriz de NxM
        df_embedding = dfs_embeddings[code_region - 1]
        merged = pd.merge(random_weighted_sample, df_embedding, left_index=True, right_index=True)
        values = list(merged.embeddings)
        mx = get_matrix(values)
        v = np.array(mx)

        #sacar la distancia promedio de este iteración
        avg_distance_i = get_avg_distance(v)        

        if current_distance > avg_distance_i:
            results['lower'] += 1

        elif current_distance < avg_distance_i:
            results['higher'] += 1
            
        elif current_distance < avg_distance_i:
            results['equal'] += 1
        
        results['sum'] += avg_distance_i
        
    return results


# 1. Organismos

In [42]:
region_codes = {'Region Metropolitana de Santiago':13,
                'Region de Antofagasta':2,
                'Region de Arica y Parinacota':15,
                'Region de Atacama':3,
                'Region de Aysen':11,
                'Region de Coquimbo':4,
                'Region de La Araucania':9,
                'Region de Los Lagos':10,
                'Region de Los Rios':14,
                'Region de Magallanes':12,
                'Region de Nuble':16,
                'Region de OHiggins':6,
                'Region de Tarapaca':1,
                'Region de Valparaiso':5,
                'Region del Biobio':8,
                'Region del Maule':7}

In [43]:
def get_matrix(list_embeddings):
    tx_list_embeddings = []
    for i in list_embeddings:
        new_list = i.split()
        new_list2 = []
        for j in new_list:
            try:
                j = j.replace('\n','')
                j = j.replace('[','')
                j = j.replace(']','')
                tmp = float(j)
                new_list2.append(tmp)
            except:
                pass

        tx_list_embeddings.append(new_list2)
    return tx_list_embeddings
    

In [47]:
#Loop para cada organismo

dict_organismos = {}
organismos = set(df_whole['organismo_codigo'])

for organismo in organismos:
    
    #Se filtra los empleados por organismo
    df_organismo = df_whole[df_whole['organismo_codigo'] == organismo]    

    #Número de personas en el organismo
    total = len(df_organismo['Paterno'])    
    
    #Sacamos la region
    region = df_organismo['region']
    regiones = set(region)
    lenght = len(regiones)
    
    
    try:
        if lenght == 1 and total > 1:
            regiones = list(val for val in regiones)
            region = regiones[0]
            region = unicodedata.normalize("NFKD",region).encode("ascii","ignore").decode("ascii")
            code_region = region_codes[region]  
            population = df_paternos_region[df_paternos_region['REG'] == code_region] 
            population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()

            #Distancia actual
            df_embedding = dfs_embeddings[code_region-1]
            df_organismo_distancias = pd.merge(df_organismo, df_embedding, left_on='Paterno', right_index=True)
            values_column = list(df_organismo_distancias.embeddings)
            matrix = get_matrix(values_column)
            np_matrix = np.array(matrix)
            current_distance = get_avg_distance(np_matrix)

            #MonteCarlo
            output = MonteCarlo(population, total, 'PORCENTAJE', iters, code_region, current_distance)
            output['distance'] = current_distance
            dict_organismos[organismo] = output
            print(organismo, output)

        #Composición toda la poblacion, para los embedding usamos la RM
        elif total > 1:
            code_region = 13
            population = df_paternos_total
            population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
            

            #Distancia actual
            df_embedding = dfs_embeddings[code_region-1]
            df_organismo_distancias = pd.merge(df_organismo, df_embedding, left_on='Paterno', right_index=True)
            values_column = list(df_organismo_distancias.embeddings)
            matrix = get_matrix(values_column)
            np_matrix = np.array(matrix)
            current_distance = get_avg_distance(np_matrix)

            #MonteCarlo
            output = MonteCarlo(population, total, 'PORCENTAJE', iters, code_region, current_distance)
            output['distance'] = current_distance
            dict_organismos[organismo] = output
            print(organismo, output)

        else:
            print(organismo, ' solo un empleado')
    
    except Exception as e:
        print(organismo, ' da error ')
                    


MA021 {'higher': 45, 'lower': 55, 'equal': 0, 'sum': 534.17225969644, 'distance': 5.382946732451738}
AM010 {'higher': 3, 'lower': 97, 'equal': 0, 'sum': 710.999332062647, 'distance': 7.131116644086635}
MU090 {'higher': 79, 'lower': 21, 'equal': 0, 'sum': 562.3428483005827, 'distance': 5.590559529624641}
AO006 {'higher': 38, 'lower': 62, 'equal': 0, 'sum': 710.9880201441231, 'distance': 7.11813516606952}
AP001 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 711.2713003515056, 'distance': 7.009470951935446}
AH003 {'higher': 97, 'lower': 3, 'equal': 0, 'sum': 709.99839405118, 'distance': 6.775552448810153}
MU084 {'higher': 2, 'lower': 98, 'equal': 0, 'sum': 709.1256782246589, 'distance': 7.138099448636916}
AR002 {'higher': 79, 'lower': 21, 'equal': 0, 'sum': 711.3110644425574, 'distance': 7.071742681541362}
CM013 {'higher': 99, 'lower': 1, 'equal': 0, 'sum': 519.7373119151622, 'distance': 5.161745829688263}
AN001 {'higher': 91, 'lower': 9, 'equal': 0, 'sum': 711.1946466420562, 'distance': 

CM008 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 531.6797737717916, 'distance': 5.269563271008128}
MU130 {'higher': 83, 'lower': 17, 'equal': 0, 'sum': 414.34152720319355, 'distance': 4.073872904074425}
CM043 {'higher': 86, 'lower': 14, 'equal': 0, 'sum': 447.96114432576525, 'distance': 4.4693854893084115}
CM019 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 578.8347422260454, 'distance': 5.736832953326505}
AP009 {'higher': 96, 'lower': 4, 'equal': 0, 'sum': 549.573563736213, 'distance': 5.445590900292261}
MU339 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 509.01509769268705, 'distance': 5.198085300789759}
CM040 {'higher': 24, 'lower': 76, 'equal': 0, 'sum': 709.567610034621, 'distance': 7.111649520491337}
MU051 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 531.7524486075133, 'distance': 5.263217366585349}
MU078 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 549.1640104645302, 'distance': 5.436186687088104}
MU217 {'higher': 91, 'lower': 9, 'equal': 0, 'sum': 431.75605424599155, 

MU121 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 709.4116288630954, 'distance': 7.173405018778099}
AO109 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 709.5684585430615, 'distance': 7.040759239183286}
CM002 {'higher': 55, 'lower': 45, 'equal': 0, 'sum': 464.97385571212027, 'distance': 4.649222479674889}
MU014 {'higher': 64, 'lower': 36, 'equal': 0, 'sum': 709.3314398945441, 'distance': 7.080946059722941}
MU171 {'higher': 27, 'lower': 73, 'equal': 0, 'sum': 465.0943371994068, 'distance': 4.659324720315384}
AO082 {'higher': 73, 'lower': 27, 'equal': 0, 'sum': 414.33081091949276, 'distance': 4.136298086202904}
AL002 {'higher': 16, 'lower': 84, 'equal': 0, 'sum': 709.8896947892393, 'distance': 7.201700665389881}
MU200 {'higher': 84, 'lower': 16, 'equal': 0, 'sum': 448.0846455074096, 'distance': 4.463772154619596}
AO066 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 520.7497171358158, 'distance': 5.138616550471162}
MA024 {'higher': 60, 'lower': 40, 'equal': 0, 'sum': 522.0599023076035,

AO002 {'higher': 26, 'lower': 74, 'equal': 0, 'sum': 709.9369885479127, 'distance': 7.124737975943508}
AD008 {'higher': 49, 'lower': 51, 'equal': 0, 'sum': 711.376867691083, 'distance': 7.115833071912413}
AB082 {'higher': 79, 'lower': 21, 'equal': 0, 'sum': 531.669820795963, 'distance': 5.281348168271119}
MU264 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 509.2474282496316, 'distance': 5.269850697360036}
AO009 {'higher': 20, 'lower': 80, 'equal': 0, 'sum': 709.2168610456364, 'distance': 7.106324728727566}
AL003 {'higher': 51, 'lower': 49, 'equal': 0, 'sum': 711.4674238602754, 'distance': 7.115948438563135}
AO108 {'higher': 70, 'lower': 30, 'equal': 0, 'sum': 709.358766101595, 'distance': 7.074650718401574}
AE012 {'higher': 74, 'lower': 26, 'equal': 0, 'sum': 710.2306710666971, 'distance': 7.029546722395716}
CF004 {'higher': 12, 'lower': 88, 'equal': 0, 'sum': 448.522268768953, 'distance': 4.5489255202489804}
MU021 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 434.90292314840025, 'di

MU079 {'higher': 1, 'lower': 99, 'equal': 0, 'sum': 520.3852922208129, 'distance': 5.315198812251975}
MU165 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 531.6935766375267, 'distance': 5.382747364898987}
AP008 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 531.7086647080523, 'distance': 5.197287864607769}
MU308 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 709.3589936559347, 'distance': 7.121607075766582}
AO048 {'higher': 98, 'lower': 2, 'equal': 0, 'sum': 562.2360867128806, 'distance': 5.5817946184614815}
CM022 {'higher': 65, 'lower': 35, 'equal': 0, 'sum': 709.6479015036788, 'distance': 7.088596715054993}
MU138 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 549.2497495110124, 'distance': 5.400320270519156}
AL005 {'higher': 34, 'lower': 66, 'equal': 0, 'sum': 711.1831510200823, 'distance': 7.117375278356304}
AL010 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 711.3780816581518, 'distance': 6.781673621051031}
MU049 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 519.2719234206568, 'd

AO046 {'higher': 91, 'lower': 9, 'equal': 0, 'sum': 532.1870509428393, 'distance': 5.2881652900174085}
AP010 {'higher': 99, 'lower': 1, 'equal': 0, 'sum': 562.181501238743, 'distance': 5.579909279799613}
MU234 {'higher': 99, 'lower': 1, 'equal': 0, 'sum': 413.92424676677274, 'distance': 4.077157363957635}
AS002 {'higher': 35, 'lower': 65, 'equal': 0, 'sum': 707.9950362129229, 'distance': 7.110143974477974}
MA036 {'higher': 54, 'lower': 46, 'equal': 0, 'sum': 563.2792655506252, 'distance': 5.610910552911989}
MU164 {'higher': 51, 'lower': 49, 'equal': 0, 'sum': 531.5649256418183, 'distance': 5.315471890917915}
MU283 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 549.2337806988469, 'distance': 5.4712097847885826}
AO012 {'higher': 99, 'lower': 1, 'equal': 0, 'sum': 709.3819886306705, 'distance': 7.055312436618221}
MA053 {'higher': 78, 'lower': 22, 'equal': 0, 'sum': 709.0812413685228, 'distance': 6.8750857197761}
AB089 {'higher': 52, 'lower': 48, 'equal': 0, 'sum': 414.25513235332653, 'dis

MU095 {'higher': 91, 'lower': 9, 'equal': 0, 'sum': 519.6619540210643, 'distance': 5.153891441345956}
MU270 {'higher': 95, 'lower': 5, 'equal': 0, 'sum': 448.13556099034247, 'distance': 4.4646839126818385}
AQ001 {'higher': 71, 'lower': 29, 'equal': 0, 'sum': 711.1281326827414, 'distance': 7.097275307415784}
MU322 {'higher': 13, 'lower': 87, 'equal': 0, 'sum': 465.0428136552004, 'distance': 4.663584464582507}
MU303 {'higher': 97, 'lower': 3, 'equal': 0, 'sum': 531.8209627821302, 'distance': 5.283740440063164}
AN002 {'higher': 85, 'lower': 15, 'equal': 0, 'sum': 711.6100478335554, 'distance': 7.06605618586837}
MU137 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 562.5381209907406, 'distance': 5.566129724124521}
AO008 {'higher': 49, 'lower': 51, 'equal': 0, 'sum': 709.2454334702228, 'distance': 7.094254964907684}
CM042 {'higher': 10, 'lower': 90, 'equal': 0, 'sum': 709.5367784304566, 'distance': 7.122609608933291}
AM005 {'higher': 60, 'lower': 40, 'equal': 0, 'sum': 710.9737796283915, 'di

MU132 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 709.5175867839675, 'distance': 7.218733120906334}
BC002 {'higher': 99, 'lower': 1, 'equal': 0, 'sum': 710.4142634326785, 'distance': 6.653802535843525}
AH015 {'higher': 69, 'lower': 31, 'equal': 0, 'sum': 712.85852906227, 'distance': 7.063944280813322}
AO057 {'higher': 0, 'lower': 100, 'equal': 0, 'sum': 508.5943519691591, 'distance': 5.232108813522273}
MU223 {'higher': 82, 'lower': 18, 'equal': 0, 'sum': 531.8188675097385, 'distance': 5.298517188276316}
AO074 {'higher': 81, 'lower': 19, 'equal': 0, 'sum': 709.2902576948175, 'distance': 7.07175803050168}
MU037 {'higher': 52, 'lower': 48, 'equal': 0, 'sum': 549.2212373039899, 'distance': 5.486983563671259}
MU003 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 562.601583814626, 'distance': 5.147771683504503}
AL004 {'higher': 7, 'lower': 93, 'equal': 0, 'sum': 710.8746976825166, 'distance': 7.162000867939057}
AL009 {'higher': 74, 'lower': 26, 'equal': 0, 'sum': 711.4998376969301, 'distan

CO005 {'higher': 72, 'lower': 28, 'equal': 0, 'sum': 711.2273095926482, 'distance': 7.023361331298822}
AO104 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 486.0431458513925, 'distance': 4.786529386914214}
AK001 {'higher': 51, 'lower': 49, 'equal': 0, 'sum': 710.4665271702637, 'distance': 7.106425969349625}
AL001 {'higher': 94, 'lower': 6, 'equal': 0, 'sum': 710.8025911070041, 'distance': 7.006060223809035}
MU296 {'higher': 100, 'lower': 0, 'equal': 0, 'sum': 519.698212090544, 'distance': 5.094283038261532}
AM002 {'higher': 32, 'lower': 68, 'equal': 0, 'sum': 711.0266203638951, 'distance': 7.130151776379342}
AB078 {'higher': 80, 'lower': 20, 'equal': 0, 'sum': 434.3959689005202, 'distance': 4.321031808893993}


In [48]:
dict_organismos

{'MA021': {'higher': 45,
  'lower': 55,
  'equal': 0,
  'sum': 534.17225969644,
  'distance': 5.382946732451738},
 'AM010': {'higher': 3,
  'lower': 97,
  'equal': 0,
  'sum': 710.999332062647,
  'distance': 7.131116644086635},
 'MU090': {'higher': 79,
  'lower': 21,
  'equal': 0,
  'sum': 562.3428483005827,
  'distance': 5.590559529624641},
 'AO006': {'higher': 38,
  'lower': 62,
  'equal': 0,
  'sum': 710.9880201441231,
  'distance': 7.11813516606952},
 'AP001': {'higher': 100,
  'lower': 0,
  'equal': 0,
  'sum': 711.2713003515056,
  'distance': 7.009470951935446},
 'AH003': {'higher': 97,
  'lower': 3,
  'equal': 0,
  'sum': 709.99839405118,
  'distance': 6.775552448810153},
 'MU084': {'higher': 2,
  'lower': 98,
  'equal': 0,
  'sum': 709.1256782246589,
  'distance': 7.138099448636916},
 'AR002': {'higher': 79,
  'lower': 21,
  'equal': 0,
  'sum': 711.3110644425574,
  'distance': 7.071742681541362},
 'CM013': {'higher': 99,
  'lower': 1,
  'equal': 0,
  'sum': 519.7373119151622,


In [49]:
#Guardar en csv resultados
nombre_archivo = 'resultados/1-node2vec_organismos-{0}.csv'.format(iters)
data = [] 
columns = []

for key, value in dict_organismos.items():
    data.append(value)
    columns.append(key)
    
total = pd.DataFrame(data, index = columns) 
total.to_csv(nombre_archivo)

In [226]:
merged_orgs = pd.merge(total, df_organism_, left_index=True, right_on='Codigo_org')


In [228]:
target_csv = merged_orgs[['Organismo','Codigo_org','higher', 'lower','equal', 'sum', 'distance']]
target_csv

,Organismo,Codigo_org,higher,lower,equal,sum,distance
634,Municipalidad de Los Vilos,MU157,0,100,0,495.67,5.14
565,Municipalidad de Antofagasta,MU009,57,43,0,507.35,5.05
959,Servicio Local de Educación Pública Colchagua,AJ026,37,63,0,498.08,5.19
486,Municipalidad de Padre Hurtado,MU193,64,36,0,556.69,5.55
874,Hospital Víctor Ríos Ruiz de Los Angeles,AO060,92,8,0,505.03,5.03
...,...,...,...,...,...,...,...
812,Municipalidad de Sagrada Familia,MU278,3,97,0,480.75,4.88
149,Municipalidad de Santiago,MU308,11,89,0,556.52,5.58
489,Hospital Base de Valdivia,AO093,100,0,0,504.83,4.99
888,Municipalidad de Tucapel,MU329,57,43,0,505.35,5.05


In [229]:
#Guardar en target resultados
nombre_archivo = 'resultados/node2vec-resultados-{0}.csv'.format(iters)
data = [] 
columns = []
    
target_csv.to_csv(nombre_archivo)

In [231]:
merged

,CANTIDAD,PORCENTAJE,Unnamed: 0,embeddings
GONZALEZ,444542,0.02,4,[ 0.07639981 -0.14802025 0.8610503 1.103278...
RODRIGUEZ,147724,0.01,19,[ 0.2454979 1.1547126 0.3852844 -0.331643...
ROJAS,249660,0.01,2,[-0.24600093 -0.08101191 -0.2091129 0.621202...
ARAYA,136972,0.01,3,[ 0.15365447 0.3810146 -0.9008908 -0.427536...
GALLEGOS,18218,0.00,428,[ 0.53985447 -0.02775288 -1.4175067 -0.726242...
CASANOVA,11048,0.00,1338,[ 1.0965875 0.64144856 -0.654297 0.217552...
MARTINEZ,152469,0.01,30,[ 0.9419716 -0.7053095 -0.25432888 0.217045...
ARIAS,39727,0.00,208,[-0.95381457 1.1875159 0.4057423 -0.555255...


# 2. Area

In [ ]:
#Loop para cada organismo

dict_organismos = {}
organismos = set(df_whole['organismo_codigo'])

for organismo in organismos:
    
    #Se filtra los empleados por organismo
    df_organismo = df_whole[df_whole['organismo_codigo'] == organismo]    

    #Número de personas en el organismo
    total = len(df_organismo['Paterno'])    
    
    #Sacamos la region
    region = df_organismo['region']
    regiones = set(region)
    lenght = len(regiones)
    
    
    try:
        if lenght == 1 and total > 1:
            regiones = list(val for val in regiones)
            region = regiones[0]
            region = unicodedata.normalize("NFKD",region).encode("ascii","ignore").decode("ascii")
            code_region = region_codes[region]  
            population = df_paternos_region[df_paternos_region['REG'] == code_region] 
            population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()

            #Distancia actual
            df_embedding = dfs_embeddings[code_region-1]
            df_organismo_distancias = pd.merge(df_organismo, df_embedding, left_on='Paterno', right_index=True)
            values_column = list(df_organismo_distancias.embeddings)
            matrix = get_matrix(values_column)
            np_matrix = np.array(matrix)
            current_distance = get_avg_distance(np_matrix)

            #MonteCarlo
            output = MonteCarlo(population, total, 'PORCENTAJE', iters, code_region, current_distance)
            output['distance'] = current_distance
            dict_organismos[organismo] = output
            print(organismo, output)

        #Composición toda la poblacion, para los embedding usamos la RM
        elif total > 1:
            code_region = 13
            population = df_paternos_total
            population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
            

            #Distancia actual
            df_embedding = dfs_embeddings[code_region-1]
            df_organismo_distancias = pd.merge(df_organismo, df_embedding, left_on='Paterno', right_index=True)
            values_column = list(df_organismo_distancias.embeddings)
            matrix = get_matrix(values_column)
            np_matrix = np.array(matrix)
            current_distance = get_avg_distance(np_matrix)

            #MonteCarlo
            output = MonteCarlo(population, total, 'PORCENTAJE', iters, code_region, current_distance)
            output['distance'] = current_distance
            dict_organismos[organismo] = output
            print(organismo, output)

        else:
            print(organismo, ' solo un empleado')
    
    except as e:
        print(organismo, ' da error ')
                    

# 2. Tipo de Contrato

# 3. Región

# 4. Deciles


In [32]:
organismos = set(df_whole['organismo_codigo'])
len(organismos)

655

In [25]:
df_whole[df_whole['organismo_codigo'] == 'CM053']

,camino,organismo_nombre,organismo_codigo,fecha_publicacion,anyo,Mes,Nombres,Paterno,Materno,Tipo cargo,...,fecha_ingreso,fecha_termino,observaciones,viaticos,enlace,activado,Tipo Estamento,descripcion_funcion,remuneracionbruta,pago_mensuales


In [20]:
set(df_organism_['Organismo'])

{'AGENCIA CHILENA DE COOPERACIÓN INTERNACIONAL PARA EL DESARROLLO (AGCID)',
 'Administradora de los Tribunales Tributarios y Aduaneros (ATTA)',
 'Agencia Nacional de Inteligencia (ANI)',
 'Agencia de Calidad de la Educación',
 'Agencia de Promoción de la Inversión Extranjera',
 'Armada de Chile',
 'Asociación Chilena de Municipalidades',
 'Asociación Chilena de Municipalidades con Farmacias Populares (ACHIFARP)',
 'Asociación Metropolitana de Municipalidades de Santiago Sur para la Gestión Ambiental y de Residuos (Msur)',
 'Asociación Municipal para la sustentabilidad ambiental AMUSA',
 'Asociación Municipalidades Provincia de Llanquihue Para Manejo Sustentable de Residuos y Gestión Ambiental',
 'Asociación Nacional de Municipalidades de Ciudades Puerto Concesionados',
 'Asociación Regional de Municipalidades de Magallanes y Antártica Chilena (AMUMAG)',
 'Asociación Regional de Municipalidades de Ñuble (ARMUÑUBLE)',
 'Asociación Regional de Municipios de Atacama',
 'Asociación de Munic

In [22]:
df_organism_[df_organism_['Organismo']== 'Corporación Municipal de Viña Del Mar']

,Codigo_org,idOrg,Organismo,Codigo_padre,Padre_org,Region,Municipalidad,direccion,telefono,url_organismo,...,tipo_cuenta,banco,url_sai,fax,Ingresa,obligadorecibir_sai,organismo_autonomo,interopera,tiene_ta,fecha_ta
680,CM053,58879,Corporación Municipal de Viña Del Mar,CM000,Corporaciones Municipales,Región de Valparaíso,VIÑA DEL MAR,NaN,NaN,http://,...,NaN,NaN,http://,NaN,No ingresa,SÍ,NO,NO,NO,NaN


In [36]:
organismos_fuera = set(df_organism_[~df_organism_.Codigo_org.isin(organismos)].Organismo)
len(organismos_fuera)


326

In [39]:
for org in organismos_fuera: print(org)

Universidad Metropolitana de Ciencias de la Educación (UMCE)
Empresa Concesionaria de Servicios Sanitarios (ECONSSA)
Fábricas y Maestranzas del Ejército (FAMAE)
Gobernación de Curicó
Empresa Portuaria Iquique
Gobernación de Huasco
Intendencia de Los Lagos
Gobernación de Copiapó
Partido Socialista de Chile PS
Universidad de La Serena
Empresa Portuaria Talcahuano San Vicente
Universidad de Magallanes
TRIBUNAL TRIBUTARIO Y ADUANERO DE LA REGIÓN DE MAGALLANES Y LA ANTÁRTICA CHILENA
TRIBUNAL TRIBUTARIO Y ADUANERO DE LA REGIÓN DE LA ARAUCANÍA
Gobernación de Cauquenes
Corporación Municipal de Punta Arenas (CORMUPA)
Intendencia del Bío Bío
Partido Pais
TRIBUNAL TRIBUTARIO Y ADUANERO DE LA REGIÓN DE ANTOFAGASTA
Empresa Nacional de Aeronáutica de Chile
Intendencia de Atacama
Municipalidad de San Rafael
Asociación de Municipalidades Red Nacional de Comunas Mágicas de Chile
Televisión Nacional de Chile (TVN)
Municipalidad de Longaví
Municipalidad de Copiapó
Corporación Nacional del Cobre de Chile 